
# Tutorial: The Flaw of a Single Split: The "Luck of the Draw" Problem

Sejauh ini, kita telah menggunakan `train_test_split` untuk membagi data kita, melatih model pada set pelatihan, dan mengevaluasinya pada set pengujian. Pendekatan ini sangat bagus dan merupakan langkah pertama yang benar dalam setiap proyek *machine learning*.

Namun, pendekatan ini memiliki satu kelemahan tersembunyi: hasil evaluasi kita bisa sangat **bergantung pada keberuntungan** dari bagaimana data tersebut dibagi secara acak.

Di notebook ini, kita akan membuktikan secara praktis bagaimana satu kali pembagian data bisa memberikan estimasi performa yang tidak stabil, dan mengapa kita memerlukan teknik yang lebih robust seperti *Cross-Validation*.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Menjelaskan mengapa evaluasi yang hanya didasarkan pada satu `train_test_split` bisa menyesatkan.
* Mendemonstrasikan melalui kode bagaimana mengubah `random_state` dapat menghasilkan skor performa yang berbeda secara signifikan.
* Memahami intuisi di balik "luck of the draw" dalam pembagian data.
* Menyadari perlunya metode evaluasi yang lebih stabil dan andal.




---
### 2. Setup: Skenario Prediksi Harga Rumah

Kita akan menggunakan dataset California Housing dari Scikit-learn. Tujuannya adalah memprediksi harga rumah (`MedHouseVal`) berdasarkan berbagai fitur.



In [5]:
import sys, sklearn, pandas
print("python:", sys.version)
print("scikit-learn:", sklearn.__version__)
print("pandas:", pandas.__version__)

python: 3.11.14 (main, Dec  8 2025, 23:47:31) [GCC 14.2.0]
scikit-learn: 1.8.0
pandas: 2.3.3


In [6]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# muat dataset
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target, name='MedHouseVal')

# tampilkan data
print("Fitur (X):")
print(X.head())
print("\nTarget (Y):")
print(y.head())

HTTPError: HTTP Error 403: Forbidden


---
### 3. Eksperimen: Dua Pembagian Data yang Berbeda

Sekarang, kita akan melakukan eksperimen sederhana. Kita akan melakukan proses yang sama persis dua kali:
1.  Bagi data.
2.  Latih model Regresi Linear.
3.  Evaluasi R-squared.

Satu-satunya hal yang akan kita ubah adalah `random_state` pada `train_test_split`. `random_state` memastikan bahwa pembagian acak dapat direproduksi. Dengan mengubahnya, kita mensimulasikan dua pembagian acak yang berbeda.




#### Pembagian Pertama: `random_state=42`



In [ ]:
X_train_1,X_test_1,y_train_1,y_test_1 = train_test_split(X,y,test_size=0.2,random_state=42)

model_1 = LinearRegression()
model_1.fit(X_train_1,y_train_1)

y_pred_1 = model_1.predict(X_test_1)
r2_1 = r2_score(y_test_1,y_pred_1)
print(f"R_squared untuk pembagian dengan random_state=42 : {r2_1:.4f}")


#### Pembagian Kedua: `random_state=101`



In [ ]:
X_train_2,X_test_2,y_train_2,y_test_2 = train_test_split(X,y,test_size=0.2,random_state=101)

model_2 = LinearRegression()
model_2.fit(X_train_2,y_train_2)

y_pred_2 = model_2.predict(X_test_2)
r2_2 = r2_score(y_test_2,y_pred_2)
print(f"R_squared untuk pembagian dengan random_state=101 : {r2_2:.4f}")


---
### 4. Analisis Hasil: Mengapa Skornya Berbeda?

Mari kita lihat hasilnya berdampingan:
* **Skor dengan `random_state=42`:** sekitar 0.5758
* **Skor dengan `random_state=101`:** sekitar 0.5971

**Perbedaan skornya cukup signifikan!** Ini menunjukkan bahwa performa model kita tampaknya bergantung pada "keberuntungan" pembagian data.

**Analogi:**
Bayangkan Anda ingin menilai kemampuan seorang siswa.
* **Pembagian 1 (`random_state=42`):** Secara kebetulan, set pengujian (`X_test_1`) mendapatkan beberapa contoh rumah yang sangat "sulit" atau tidak biasa, yang membuat model kesulitan memprediksinya. Hasilnya, skor R² lebih rendah.
* **Pembagian 2 (`random_state=101`):** Secara kebetulan, set pengujian (`X_test_2`) mendapatkan contoh-contoh rumah yang lebih "mudah" atau tipikal, yang lebih sesuai dengan pola yang dipelajari model. Hasilnya, skor R² lebih tinggi.

Kedua skor ini benar berdasarkan pembagiannya masing-masing, tetapi **keduanya tidak memberikan gambaran yang lengkap dan stabil** tentang seberapa baik model kita sebenarnya. Kita tidak bisa hanya memilih skor yang lebih tinggi dan mengabaikan yang lain.




---
### 5. Kesimpulan

* Evaluasi berdasarkan satu kali `train_test_split` **tidaklah robust**. Hasilnya bisa sangat bervariasi tergantung pada bagaimana data dibagi.
* Melaporkan hanya satu skor bisa menjadi terlalu **optimis** (jika kita "beruntung" mendapatkan pembagian yang mudah) atau terlalu **pesimis** (jika kita "sial").
* Kita memerlukan sebuah metode yang dapat memberikan estimasi performa yang lebih **stabil dan andal** dengan cara mengevaluasi model pada beberapa set pengujian yang berbeda.

Metode inilah yang disebut **K-Fold Cross-Validation**, yang akan menjadi topik utama di subchapter berikutnya.
